# LightGBMの推論用カスタムコンテナを構築し、SageMakerによる推論の仕組みを深く理解する

このノートブックでは、LightGBMがインストールされたカスタムコンテナ構築し、SageMaker Trainingジョブで学習後、推論を行います。
カスタムコンテナの挙動を観察し、SageMakerの推論動作について理解を深めます。

ノートブックは20分程度で実行できます。

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [1]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

In [2]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

Python 3.8.12


## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

SageMakerSDK をインポートすると、バケットが作成されます。  
sagemaker-＜region＞-＜account＞

In [3]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.110.0

Current SageMaker Python SDK Version =2.112.2


# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [4]:
import sklearn
sklearn.__version__ # 1.0.1

'1.0.1'

In [5]:
import pandas as pd
pd.__version__ # 1.3.4

'1.3.4'

## 1-1. データロード

In [6]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [7]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [9]:
# 確認
print(trainX.shape)
trainX.head()

(303, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
1,0.33983,22.0,5.86,0.0,0.431,6.108,34.9,8.0555,7.0,330.0,19.1,390.18,9.16,24.3
2,0.10469,40.0,6.41,1.0,0.447,7.267,49.0,4.7872,4.0,254.0,17.6,389.25,6.05,33.2
3,6.80117,0.0,18.10,0.0,0.713,6.081,84.4,2.7175,24.0,666.0,20.2,396.90,14.70,20.0
4,1.35472,0.0,8.14,0.0,0.538,6.072,100.0,4.1750,4.0,307.0,21.0,376.73,13.04,14.5


In [10]:
# 確認
print(valX.shape)
valX.head()

(101, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.03150,95.0,1.47,0.0,0.403,6.975,15.3,7.6534,3.0,402.0,17.0,396.90,4.56,34.9
1,0.51183,0.0,6.20,0.0,0.507,7.358,71.6,4.1480,8.0,307.0,17.4,390.07,4.73,31.5
2,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44,15.0
3,0.95577,0.0,8.14,0.0,0.538,6.047,88.8,4.4534,4.0,307.0,21.0,306.38,17.28,14.8
4,0.09604,40.0,6.41,0.0,0.447,6.854,42.8,4.2673,4.0,254.0,17.6,396.90,2.98,32.0


In [11]:
# 確認
print(testX.shape)
testX.head()

(102, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81
1,0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
2,4.64689,0.0,18.10,0.0,0.6140,6.980,67.6,2.5329,24.0,666.0,20.2,374.68,11.66
3,3.67367,0.0,18.10,0.0,0.5830,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58
4,0.29819,0.0,6.20,0.0,0.5040,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92


In [12]:
# 確認
y_test[0:5]

array([14.4, 33. , 29.8, 21.2, 46.7])

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [13]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [14]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

sagemaker-＜region＞-＜accoutid＞　を取得することができます。

In [15]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

In [16]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

sagemaker-ap-northeast-1-316134882092
ap-northeast-1
316134882092


In [17]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [18]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

In [19]:
# 確認:格納したS3のURIが返されています
print(train_s3)
print(valid_s3)

s3://sagemaker-ap-northeast-1-316134882092/demo_lightgbm/train/boston_train.csv
s3://sagemaker-ap-northeast-1-316134882092/demo_lightgbm/valid/boston_valid.csv


# 2.LightGBMカスタムコンテナの構築


学習用のカスタムコンテナの作成には大きく分けて3つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

まずはSageMakerの動作を理解するためにベースイメージ(ubuntu:16.04) + カスタムレイヤー方式を採用します。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

まずは、Dockerfileを確認します。

In [20]:
!pygmentize ./container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

ARG CONDA_DIR=/opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN apt-get update && \
    apt-get install -y --no-install-recommends \
        ca-certificates \
        cmake \
        build-essential \
        gcc \
        g++ \
        git \
        nginx \
        wget && \
    # python environment
    wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh && \
    /bin/bash Miniconda3-latest-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    export PATH="$CONDA_DIR/bin:$PATH" && \
    conda config --set always_yes yes --set changeps1 no && \
    # lightgbm
    conda install -q -y numpy scipy scikit-learn pandas flask gevent gunicorn && \
    git clone --recursive --branch stable --depth 1 https://github.com/Microsoft/LightGBM && \
 

### 解説：推論エンドポイント構築時のSageMakerの動作
SageMakerの推論エンドポイントのデプロイは、SageMaker SDKでは、deploy()メソッドで実行します。

https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html

その際に、SageMakerは以下のコマンドを実行します。

docker run < Docker image > server

今回のカスタムコンテナでは、 /opt/program に配置した serve スクリプトが実行されます。

serveスクリプトを確認してみましょう。

In [21]:
!pygmentize -l py ./container/lightgbm_regression/serve

#!/usr/bin/env python

# This file implements the scoring service shell. You don't necessarily need to modify it for various
# algorithms. It starts nginx and gunicorn with the correct configurations and then simply waits until
# gunicorn exits.
#
# The flask server is specified to be the app object in wsgi.py
#
# We set the following parameters:
#
# Parameter                Environment Variable              Default Value
# ---------                --------------------              -------------
# number of workers        MODEL_SERVER_WORKERS              the number of CPU cores
# timeout                  MODEL_SERVER_TIMEOUT              60 seconds

from __future__ import print_function
import multiprocessing
import os
import signal
import subprocess
import sys

cpu_count = multiprocessing.cpu_count()

model_server_timeout = os.environ.get('MODEL_SERVER_TIMEOUT', 60)
model_server_workers = int(os.environ.get('MODEL_SERVER_WORKERS', cpu_count))

def sigterm_handler(nginx_pid, gunicorn_

末尾の start_server() を実行しており、start_server()では以下が行われます。

* nginxの起動（Webサーバ/リバースプロキシの役割）
    * nginx.confを読み込みます。
* gunicornの起動（Applicationサーバの役割）
    * gunicornの起動コマンド引数に'wsgi:app'とあるように、wsgiモジュールwsgi.pyの、appアプリケーションを読み込みます。

nginx.confを確認します。

In [22]:
!pygmentize ./container/lightgbm_regression/nginx.conf

worker_processes 1;
daemon off; # Prevent forking


pid /tmp/nginx.pid;
error_log /var/log/nginx/error.log;

events {
  # defaults
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /var/log/nginx/access.log combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 5m;

    keepalive_timeout 5;
    proxy_read_timeout 1200s;

    location ~ ^/(ping|invocations) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_pass http://gunicorn;
    }

    location / {
      return 404 "{}";
    }
  }
}


SageMakerから受け取った /ping と /invocations リクエストを上記で設定したgunicornに渡します。
以下に記載があるように、ポート8080を利用する必要があります。

https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html

How Containers Serve Requests  
Containers need to implement a web server that responds to /invocations and /ping on port 8080.

次に、gunicornへのアプリケーションのキック用に使われるファイル wsgi.pyを確認します。

predictor.py の、appを読み込んでいることがわかります。

In [23]:
!pygmentize ./container/lightgbm_regression/wsgi.py

import predictor as myapp

# This is just a simple wrapper for gunicorn to find your app.
# If you want to change the algorithm file, simply change "predictor" above to the
# new file.

app = myapp.app


predictor.py を確認します。

flaskフレームワークを用いて、/ping, /invocations に対する処理を実装していることがわかります。

In [24]:
!pygmentize ./container/lightgbm_regression/predictor.py

# This is the file that implements a flask server to do inferences. It's the file that you will modify to
# implement the scoring for your own algorithm.

from __future__ import print_function

import os
import json
import pickle
import sys
import signal
import traceback
import io
import flask

import pandas as pd
import lightgbm as lgb

prefix = '/opt/ml/'
model_path = os.path.join(prefix, 'model')


# A singleton for holding the model. This simply loads the model and holds it.
# It has a predict function that does a prediction based on the model and the input data.

class ScoringService(object):
    model = None  # Where we keep the model when it's loaded

    @classmethod
    def get_model(cls):
        """Get the model object for this instance, loading it if it's not already loaded."""
        if cls.model == None:
            cls.model = lgb.Booster(model_file=os.path.join(model_path, 'lightgbm-regression-model.txt'))
        return cls.model

    @classmethod
    def predict(cls,

## 2-2. dockerイメージの build & push
上記で確認したカスタムコンテナをビルドします。

ビルド&pushには7分ほどかかります。

In [25]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon   25.6kB
Step 1/10 : FROM ubuntu:16.04
 ---> b6f507652425
Step 2/10 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 2a03957e9907
Step 3/10 : ARG CONDA_DIR=/opt/conda
 ---> Using cache
 ---> 230c4387e899
Step 4/10 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 79976a9fff60
Step 5/10 : RUN apt-get update &&     apt-get install -y --no-install-recommends         ca-certificates         cmake         build-essential         gcc         g++         git         nginx         wget &&     wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh &&     /bin/bash Miniconda3-latest-Linux-x86_64.sh -f -b -p $CONDA_DIR &&     export PATH="$CONDA_DIR/bin:$PATH" &&     conda config --set always_yes yes --set changeps1 no &&     conda install -q -y numpy scipy scikit-learn pandas flask gevent gunicorn &&     git clone --recursive --branch stable --depth 1 https://github.com/Microsoft/LightGBM && 

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## 2-3. 学習前設定
AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを設定します。

In [26]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

sagemaker-ap-northeast-1-316134882092
ap-northeast-1
316134882092


In [27]:
# imageURLの設定
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [28]:
# 確認
image_uri

'316134882092.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression'

In [29]:
# 学習で指定するLightGBMのハイパーパラメータを設定します。
hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

## 2-4.ローカル学習の実行
まずはローカルモードでモデルの学習を行います。
ローカルモードを利用することで、コンテナイメージのダウンロードや展開の手間を省くことができるため、コードのデバッグを行う場合に便利です。

ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [30]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

file://./data/train/boston_train.csv
file://./data/valid/boston_valid.csv


In [31]:
from sagemaker.estimator import Estimator

In [32]:
from sagemaker import get_execution_role

role = get_execution_role()

In [33]:
# 確認
role

'arn:aws:iam::316134882092:role/TeamRole'

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [34]:
local_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters
    )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [35]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

Creating 82zh3hwz6w-algo-1-utgm5 ... 
Creating 82zh3hwz6w-algo-1-utgm5 ... done
Attaching to 82zh3hwz6w-algo-1-utgm5
82zh3hwz6w-algo-1-utgm5 | Starting the training.
82zh3hwz6w-algo-1-utgm5 | Reading hyperparameters data: /opt/ml/input/config/hyperparameters.json
82zh3hwz6w-algo-1-utgm5 | hyperparameters_data: {'boosting_type': 'gbdt', 'objective': 'regression', 'num_leaves': '31', 'learning_rate': '0.05', 'feature_fraction': '0.9', 'bagging_fraction': '0.8', 'bagging_freq': '5', 'verbose': '0'}
82zh3hwz6w-algo-1-utgm5 | Found train files: ['/opt/ml/input/data/train/boston_train.csv']
82zh3hwz6w-algo-1-utgm5 | Found validation files: ['/opt/ml/input/data/validation/boston_valid.csv']
82zh3hwz6w-algo-1-utgm5 | building training and validation datasets
82zh3hwz6w-algo-1-utgm5 | Starting training...
82zh3hwz6w-algo-1-utgm5 | /opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of 

ローカルモードの学習結果についてもS3に保管されます。

s3://sagemaker-< リージョン名 >-< アカウントID >/sagemaker-lightgbm-regression-yyyy-MM-dd-HH-mm-ss-fff/

* model.tar.gz
* output.tar.gz

Trainingジョブの詳細について学びたい場合は、BlackBeltの解説もご参照ください。
https://www.youtube.com/watch?v=byEawTm4O4E

## 2-5.ローカルデプロイ

serializer : インプットデータの形式を指定します。
https://sagemaker.readthedocs.io/en/stable/v2.html

In [36]:
# 事前準備：全コンテナ停止
!docker stop $(docker ps -q)

684516fba33f


In [37]:
# 確認
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


起動中のコンテナイメージがないことを確認し、ローカルデプロイを行います。

In [38]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

Attaching to 0npgc7ojfb-algo-1-k53qe
0npgc7ojfb-algo-1-k53qe | Starting the inference server with 2 workers.
0npgc7ojfb-algo-1-k53qe | [2022-10-27 10:56:17 +0000] [10] [INFO] Starting gunicorn 20.1.0
0npgc7ojfb-algo-1-k53qe | [2022-10-27 10:56:17 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
0npgc7ojfb-algo-1-k53qe | [2022-10-27 10:56:17 +0000] [10] [INFO] Using worker: gevent
0npgc7ojfb-algo-1-k53qe | [2022-10-27 10:56:17 +0000] [12] [INFO] Booting worker with pid: 12
0npgc7ojfb-algo-1-k53qe | [2022-10-27 10:56:17 +0000] [13] [INFO] Booting worker with pid: 13
!0npgc7ojfb-algo-1-k53qe | 172.18.0.1 - - [27/Oct/2022:10:56:21 +0000] "GET /ping HTTP/1.1" 200 1 "-" "python-urllib3/1.26.8"


In [39]:
# 確認
!docker ps

CONTAINER ID   IMAGE                                                                             COMMAND   CREATED         STATUS         PORTS                                       NAMES
d68a00651122   316134882092.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression   "serve"   5 seconds ago   Up 4 seconds   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   0npgc7ojfb-algo-1-k53qe


ローカルにコンテナイメージが展開されていることが確認できました。

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

Estimatorの引数instance_typeにインスタンスタイプを指定することで、学習ジョブが発行されます。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
    hyperparameters=hyperparameters)

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

課金されるのは75秒ほどです。

## 2-8.エンドポイントにデプロイ

デプロイすると、
SageMaker は docker run <image> serveを実行します。

    
デプロイには3分ほどかかります。

In [ ]:
from sagemaker.predictor import csv_serializer

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3.推論コードの外部指定、フロントエンドはSageMakerが準備した仕組みを利用する。
推論コードを外部から指定するために、SageMaker Inference Toolkitを導入します。

https://github.com/aws/sagemaker-inference-toolkit

また、前セクションnginx, gunicorn, flaskを用いて実装したモデルサービングの仕組みはSageMaker側で準備されたものを流用します。
これは、MMS(Multi Model Server)というライブラリを導入します。

https://github.com/awslabs/multi-model-server/tree/master/docker

* SageMaker-Inference-Toolkitと、Multi Model Serverを導入する
* ビルトインコンテナ + requirements.txt, inference.pyを利用する

MMSの利用については、以下のサンプルコードも参照ください。

https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/multi_model_bring_your_own


## 3-1.Dockerfileの確認

まずは、利用するDockerfileを確認します。
MMSに必要なJavaをインストールし、MMSとinference-toolkitをインストールしています。

lightgbmはrequirements.txtでインストールを試みるため、Dockerfileには記載していません。（記載することも可能）

In [ ]:
!pygmentize ./container_sminftoolkit/Dockerfile

## 3-2.エントリポイントを確認

SageMakerSDKにてdeploy()を実行した際の

docker run \<image> server

で実行される、ENTRYPOINTを確認します。

dockerfile

これは、以下に該当する。

3.Implement a serving entrypoint, which starts the model server.


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

start_model_server()は、引数指定しない場合、

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

を指定。これは、inference-toolkitのハンドラサービスである。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py



ハンドラサービスが、Transformer()を作り、そのなかで、推論ハンドラが作られている。

DefaultHandlerService -> Transformer -> DefaultInferenceHandler

https://github.com/aws/sagemaker-inference-toolkit/blob/3774c1a0fb4408cfa95333b75d6e30a376bffa52/src/sagemaker_inference/transformer.py


In [ ]:
!pygmentize ./container_sminftoolkit/dockerd-entrypoint.py

start_model_server()は引数指定しない場合、
inference-toolkitのTransform()が作られる。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

より、

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py

__init__にて、Trransformer()が実行

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/transformer.py

Transform()において、inference-toolkitのDefaultInferenceHandlerが利用される。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_inference_handler.py

よって、このdockerd-entrypoint.pyが最小構成となる。

## 解説
ハンドラサービスと推論ハンドラがある。

ハンドラサービスは、以下に該当する。

2.Implement a handler service that is executed by the model server.

モデルの推論ハンドラは、以下に該当する。

1.Implement an inference handler, which is responsible for loading the model and providing input, predict, and output functions. 


2.のハンドラサービスから、1.の推論ハンドラがロードされる。推論ハンドラはinference-toolkitで用意したものを使ってもよい。

In [ ]:
#!pygmentize ./container_sminftoolkit/model_handler.py ### 最小構成には不要

build&pushには3分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
#algorithm_name=demo-sagemaker-multimodel
algorithm_name=demo-sagemaker-inftoolkit

#cd container
cd container_sminftoolkit

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

* dockerd-entrypoint.py が実行され、サーバーの起動を試みる。
    * サーバー起動の際に必要はハンドラーは、odel-handler.pyに記載されている。
    


## ローカルにエンドポイントをデプロイ
モデルは前のセクションで作成したLGBMモデル

* ソースも指定する
* LGBMはrequirements.txtでインストールする

In [ ]:
container_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/demo-sagemaker-inftoolkit:latest'

In [ ]:
container_uri

In [ ]:
### 2.8の学習ジョブで構築したモデルを利用する
#est_lightgbm.model_data

### ローカル学習で構築したモデルを利用する場合
model_data=local_lightgbm.model_data

In [ ]:
!docker ps

In [ ]:
#全コンテナ停止
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
from sagemaker.predictor import RealTimePredictor

lgb_model = sagemaker.model.Model(#est_xgb.image_uri, # XGBoostビルトインコンテナのURI
                                  container_uri,
                                  model_data=model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                  #entry_point='./src_builtin_container_serve/inference.py'
                                 )

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                      )

In [ ]:
!docker ps

In [ ]:
#!docker stop f380dc891702

In [ ]:
#!docker ps

## 推論実施

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(predicted)

In [ ]:
print(type(predicted))
predicted

## (option)返り値をstr以外で受け取りには
Deserializerの説明

現在は、この動画にあるように、SageMakerSDKを使っているため、
deserializerがxxxが使われます。

指定することで、numpy_arrayなどで受け取ることができます。

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

deploy()の中で、deserializerを指定します。

In [ ]:
!docker stop $(docker ps -q)

In [40]:
from sagemaker.deserializers import PandasDeserializer

In [42]:
#PandasDeserializer.deserialize([1,2,3], “text/csv”,)
PandasDeserializer.deserialize(stream=[1,2,3], content_type='application/json',)

TypeError: deserialize() missing 1 required positional argument: 'self'

In [43]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                       deserializer=PandasDeserializer
                                      )

NameError: name 'lgb_model' is not defined

https://aws.amazon.com/jp/blogs/aws/amazon-sagemaker-serverless-inference-machine-learning-inference-without-worrying-about-servers/

In [44]:
input_jsonlines = [
    {"features": ["I love this product!"]},
    {"features": ["OK, but not great."]},
    {"features": ["This is not the right product."]},
]

In [45]:
from sagemaker.serializers import JSONLinesSerializer

In [46]:
JSONLinesSerializer().serialize(input_jsonlines)

'{"features": ["I love this product!"]}\n{"features": ["OK, but not great."]}\n{"features": ["This is not the right product."]}'

In [47]:
### CSVフォーマットをシリアライズする場合

from sagemaker.serializers import CSVSerializer

input_csv = [['a1','a2'],
             ['b1','b2'],
             ['c1','c2']]

serialized = CSVSerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

CSVSerializer().serialize(input_csv)

<class 'str'>
a1,a2
b1,b2
c1,c2


'a1,a2\nb1,b2\nc1,c2'

In [48]:
### CSVフォーマットをシリアライズする場合

from sagemaker.serializers import CSVSerializer

input_csv = 'a,b,c,d,e'

serialized = CSVSerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

CSVSerializer().serialize(input_csv)

<class 'str'>
a,b,c,d,e


'a,b,c,d,e'

In [49]:
### CSVフォーマットをシリアライズする場合

from sagemaker.serializers import CSVSerializer

input_csv = [[1,2],
             [3,4],
             [5,6]]

serialized = CSVSerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

CSVSerializer().serialize(input_csv)

<class 'str'>
1,2
3,4
5,6


'1,2\n3,4\n5,6'

In [50]:
sagemaker.serializers.NumpySerializer

sagemaker.serializers.NumpySerializer

In [51]:
import numpy as np
np.array([1,2,3,4])

array([1, 2, 3, 4])


NumpyArrayにシリアライズ
https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.NumpySerializer

In [52]:
### numpyArrayフォーマットにシリアライズする場合

from sagemaker.serializers import NumpySerializer

input_csv = [[1,2],
             [3,4],
             [5,6]]


input_csv = [1,2,3,4,5,6]

input_csv = [['a1','a2'],
             ['b1','b2'],
             ['c1','c2']]
input_csv = np.array([1,2,3,4])

serialized = NumpySerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

NumpySerializer().serialize(input_csv)

<class 'bytes'>
b"\x93NUMPY\x01\x00v\x00{'descr': '<i8', 'fortran_order': False, 'shape': (4,), }                                                            \n\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00"


b"\x93NUMPY\x01\x00v\x00{'descr': '<i8', 'fortran_order': False, 'shape': (4,), }                                                            \n\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00"

### デシリアライザについて

SageMakerSDKを使ってデプロイすると、SerializerとDeserializerはnumpy arrayが設定される。

* 予測結果もnupy array型とする。
    * LightGBMのpredict()返り値はarray型：https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier.predict
* output_fnはnumpy_arrayのデータストリームを作る
* Deserializerで、inputはストリーム型で、numpyArray型（もしくはPandas DataFrame）で受け取る


list
https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.CSVDeserializer

Numpy ndarray
https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.NumpyDeserializer

Pandas DataFrame
https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.PandasDeserializer


In [53]:
CSVSerializer().serialize(pred)

NameError: name 'pred' is not defined

In [54]:
from sagemaker.deserializers import PandasDeserializer

#pred = [[1,2],
#             [3,4],
#             [5,6]]


pred = [1,2,3,4,5,6]
pred = 'a,b,c,d,e,f'

#pred = [['a1','a2'],
#             ['b1','b2'],
#             ['c1','c2']]

#pred = np.array([1,2,3,4])

pred = {
  "predictions": [{
    "closest_cluster": 5,
    "distance_to_cluster": 36.5
  }]
}

print(type(pred))

#deserialized = PandasDeserializer().deserialize(pred, 'application/json')
#deserialized = PandasDeserializer(accept='text/csv').deserialize(pred, 'text/csv')
#deserialized = PandasDeserializer().deserialize(pred, 'application/json')
deserialized = PandasDeserializer().deserialize(pred, 'text/csv')
#deserialized = PandasDeserializer().deserialize(CSVSerializer().serialize(pred), 'text/csv')


print(type(deserialized))
print(deserialized)

#NumpySerializer().serialize(input_csv)

<class 'dict'>


ValueError: Invalid file path or buffer object type: <class 'dict'>

botocore.response.StreamingBody()を作って試す

# デシリアライザがうまくいくケース ===================

In [55]:
import botocore
import json
from io import BytesIO

In [56]:
# 返却したいオブジェクト
body_json = {
    "aaa": 3,
    "bbb": [
        {
            "ccc": "ddd"
        }
    ]
}

# エンコード。(encode()はデフォルトでutf-8。)
body_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))

In [57]:
#deserialized = PandasDeserializer().deserialize(body, 'text/csv')
deserialized = PandasDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット


In [58]:
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'pandas.core.frame.DataFrame'>
   aaa             bbb
0    3  {'ccc': 'ddd'}


,aaa,bbb
0,3,{'ccc': 'ddd'}


# END: デシリアライザがうまくいくケース ===================

In [59]:
body_encoded = json.dumps(body_json).encode()

In [60]:
print(type(body_encoded))
print(body_encoded)

<class 'bytes'>
b'{"aaa": 3, "bbb": [{"ccc": "ddd"}]}'


In [61]:
body

In [62]:
import encoders

ModuleNotFoundError: No module named 'encoders'

In [63]:
pred = np.array([[1],[2],[3],[4],[5]]) ### LGBM予測の出力とする
print(type(pred))
print(pred)
pred.shape

<class 'numpy.ndarray'>
[[1]
 [2]
 [3]
 [4]
 [5]]


(5, 1)

In [64]:
print(type(pred.tobytes()))
print(pred.tobytes())

<class 'bytes'>
b'\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00'


In [65]:
body = botocore.response.StreamingBody(BytesIO(pred.tobytes()),len(pred.tobytes()))

In [66]:
body

In [67]:
NumpyDeserializer().deserialize(body, 'application/x-npy') ### SageMakerSDKの場合これが使われる。
#NumpyDeserializer().deserialize(body, 'application/json')

NameError: name 'NumpyDeserializer' is not defined

## NumpySeriarizerからDeserializerを使う

inputは同じくndarray

In [73]:
from sagemaker.deserializers import NumpyDeserializer

In [74]:
pred = np.array([[1],[2],[3],[4],[5]]) ### LGBM予測の出力とする
print(type(pred))
print(pred)
pred.shape

<class 'numpy.ndarray'>
[[1]
 [2]
 [3]
 [4]
 [5]]


(5, 1)

In [75]:
NumpySerializer().serialize(pred)

b"\x93NUMPY\x01\x00v\x00{'descr': '<i8', 'fortran_order': False, 'shape': (5, 1), }                                                          \n\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00"

In [76]:
NumpyDeserializer().deserialize(NumpySerializer().serialize(pred), 'application/x-npy') ### SageMakerSDKの場合これが使われる。

AttributeError: 'bytes' object has no attribute 'close'

In [ ]:
from sagemaker.deserializers import NumpyDeserializer

In [ ]:
pred_str = pred.astype('str')

In [ ]:
pred_str

In [ ]:
# deserializer()のインプットのための、ストリームを作成
pred = np.array([1,2,3,4,5])

#encorders.encode(pred, 'utf-8')
#body = botocore.response.StreamingBody(BytesIO(pred),len(pred))
#body = botocore.response.StreamingBody(BytesIO(pred_str),len(pred_str))

body = botocore.response.StreamingBody(pred.tobytes(),len(pred.tobytes()))


#deserialized = PandasDeserializer().deserialize(body, 'application/json')
#deserialized = PandasDeserializer().deserialize(body, 'text/csv')
#deserialized = PandasDeserializer().deserialize(pred, 'application/x-npy') ### PandasDeserializer()は、CSVとJSONのみ
#deserialized = NumpyDeserializer().deserialize(body, 'application/x-npy')
deserialized = NumpyDeserializer().deserialize(body, 'text/csv')


In [ ]:
body

In [ ]:
StreamingBody(io.BytesIO(encoded_content),
                         len(encoded_content)) 

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

# (optional) XGBoostコンテナで、LGBMの推論を実施する

LGBMのカスタムコンテナも存在する
< URL >
    

In [ ]:
xgb_container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [ ]:
xgb_container_uri

In [ ]:
from sagemaker.predictor import RealTimePredictor

lgb_model = sagemaker.model.Model(xgb_container_uri, # XGBoostビルトインコンテナのURI
                                  model_data=est_lightgbm.model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                 )

In [ ]:
!docker ps

In [ ]:
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                       #deserializer=None, 
                                      )

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(predicted)

# END of Containts =======================

# 後片付け

# 参考

## （optional）4. カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


## 4-2. 推論実施

### 4-2-1.デプロイ

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy


デプロイの際に、ソースコードを指定するにはどうしたらいいのか？

https://www.youtube.com/watch?v=sngNd79GpmE&t=596s


ポイント：あらためて、Estimatorを定義する必要がある。

### serve用のファイルは、.py かつ、作法に従う必要がある。

MMSは.pyを扱うように設計されているため。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

## エラー

RuntimeError: Model /opt/ml/model/model.tar.gz cannot be loaded:


6o0805unb3-algo-1-k8ugv | [2022-10-16 02:25:50 +0000] [19] [ERROR] Exception in worker process  
6o0805unb3-algo-1-k8ugv | Traceback (most recent call last):  
6o0805unb3-algo-1-k8ugv |   File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/algorithm_mode/serve_utils.py", line 175, in get_loaded_booster  
6o0805unb3-algo-1-k8ugv |     booster = pkl.load(open(full_model_path, "rb"))  
6o0805unb3-algo-1-k8ugv | _pickle.UnpicklingError: invalid load key, '\x1f'.  

## 原因
lightgbm-regression-model.txtなので、pklでは読み込めない。

モデルロードする関数を上書きするには？？（そもそもこれがやりたい）

https://github.com/aws/sagemaker-xgboost-container/blob/master/docker/1.5-1/final/Dockerfile.cpu

# Set SageMaker entrypoints
ENV SAGEMAKER_TRAINING_MODULE sagemaker_xgboost_container.training:main  
ENV SAGEMAKER_SERVING_MODULE sagemaker_xgboost_container.serving:main  


まず、serving.main()が実行される

https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L143

serving_env = env.ServingEnv()

で、環境変数にパラメータが読み込まれる


L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

ここで、ユーザーのモジュールが読み込まれる。

L18をみると、sagemaker_containers.beta.framework.modulesがモジュールのようだ。

from sagemaker_containers.beta.framework import (
    encoders,
    env,
    modules,
    server,
    transformer,
    worker,
)

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/beta/framework/__init__.py

sagemaker_containers.beta.frameworkはアーカイブされている。

現在はこちら。initをみると

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py



L258で、imortしている。

module = importlib.import_module(name)

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

とあるように、DEFAULT_MODULE_NAMEが読み込まれるようだ





https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L148,149: L147で読み込んだユーザーモジュールに上書きする

user_module_transformer = _user_module_transformer(user_module)  
user_module_transformer.initialize()  


L116にあるように、model_fnなどのユーザー関数に上書きされる。


def _user_module_transformer(user_module):  
    model_fn = getattr(user_module, "model_fn", default_model_fn)  
    input_fn = getattr(user_module, "input_fn", None)  
    predict_fn = getattr(user_module, "predict_fn", None)  
    output_fn = getattr(user_module, "output_fn", None)  
    transform_fn = getattr(user_module, "transform_fn", None)  

## model_fnを定義したファイルが、importされているか？

いま、そもそも環境変数に正しく情報渡せていない気がする。


https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py

L237より、

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

第二引数に指定する必要がある。

これを呼ぶのは、


https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

serving_env.module_name である。指定できているのか？


L143より

serving_env = env.ServingEnv()

これは、以下のファイル。

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L862

class ServingEnv(_Env):



https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L329には、

class _Env(_mapping.MappingMixin):


module_name = os.environ.get(_params.USER_PROGRAM_ENV, None)

とある。


L595

TrainingEnvには、

        # override base class attributes  
        if self._module_name is None:  
            self._module_name = str(sagemaker_hyperparameters.get(_params.USER_PROGRAM_PARAM, None))  
        self._user_entry_point = self._user_entry_point or sagemaker_hyperparameters.get(  
            _params.USER_PROGRAM_PARAM  
        )  
        
        
        

## USER_PROGRAM_ENVに設定できればいい？


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/parameters.py


L18

USER_PROGRAM_ENV = "SAGEMAKER_PROGRAM"  # type: str

SAGEMAKER_PROGRAMに設定できればいいようだ。

ビルトインコンテナにはどうすれば設定できるのだろうか？？

以下のYouTubeだと、boto3でEnvironment引数を使っている。

https://youtu.be/sngNd79GpmE?t=780

# デバッグのために、dockerイメージをプルして、中をみてみる。

ビルトインコンテナの中身をみるには、どうすればいいのか？

XGBoostの場合は、ローカルでbuildしていくようだ。

https://github.com/aws/sagemaker-xgboost-container

# コンテナの中に入って確認する方法
コンソールを立ち上げて、以下の流れで実行する


ディレクトリ移動
 $ cd sagemaker-xgboost-container/
 
baseコンテナをビルド
 $ docker build -t xgboost-container-base:1.5-1-cpu-py3 -f docker/1.5-1/base/Dockerfile.cpu .

finalコンテナをビルド
 $ docker build -t preprod-xgboost-container:1.5-1-cpu-py3 -f docker/1.5-1/final/Dockerfile.cpu .

構築されたイメージを確認
$ docker image ls

中に入って確認（コンテナのタグもつけて指定すること）
$ docker run -it preprod-xgboost-container:1.5-1-cpu-py3 /bin/bash

$ docker run -it 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1 /bin/bash  

   

# コンテナの中に入り、 command serveを実行してみる

# モデル利用ならうまくいくのではないか？-> OK


YouTubeのリンク先ソースより

https://github.com/aws-samples/aws-ml-jp/blob/main/sagemaker/sagemaker-inference/inference-tutorial/1_sklearn.ipynb



https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model

デプロイ

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

ValueError: Estimator is not associated with a training job

# エラー　trainingjobとの紐付け

https://stackoverflow.com/questions/63340328/how-to-define-a-sagemaker-estimator-object-using-a-pre-trained-model-and-then-de

# END of Containts ===============

# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit